## Set up the environment and Load Data

In [1]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [2]:
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
import keras
print(keras.__version__)

2.12.0


In [5]:
# !pip install git+https://github.com/yaledhlab/vggface.git
# simply use other's edited VGGFace by git

In [6]:
# !pip install keras_vggface
# install from official but need to change one import in model.py

In [5]:
from keras_vggface.vggface import VGGFace

In [8]:
# from keras_vggface.utils import preprocess_input
# already preprocessed in data preprocessing

In [6]:
from keras import Model

In [7]:
import numpy as np
train_image = np.load('train_image.npy')
train_label = np.load('train_label.npy')

In [8]:
test_image = np.load('test_image.npy')
test_label = np.load('test_label.npy')

In [9]:
train_image[0]

array([[[122.406    ,  89.2376   ,  46.813705 ],
        [127.406    ,  89.2376   ,  43.813705 ],
        [127.406    ,  89.2376   ,  43.813705 ],
        ...,
        [ 70.406    ,  60.237602 ,  39.813705 ],
        [ 70.406    ,  60.237602 ,  39.813705 ],
        [ 69.406    ,  57.237602 ,  34.813705 ]],

       [[120.406    ,  89.2376   ,  46.813705 ],
        [125.406    ,  89.2376   ,  44.813705 ],
        [125.406    ,  89.2376   ,  44.813705 ],
        ...,
        [ 76.406    ,  67.2376   ,  43.813705 ],
        [ 76.406    ,  67.2376   ,  43.813705 ],
        [ 66.406    ,  55.237602 ,  30.813705 ]],

       [[120.406    ,  89.2376   ,  46.813705 ],
        [125.406    ,  89.2376   ,  44.813705 ],
        [125.406    ,  89.2376   ,  44.813705 ],
        ...,
        [ 76.406    ,  67.2376   ,  43.813705 ],
        [ 76.406    ,  67.2376   ,  43.813705 ],
        [ 66.406    ,  55.237602 ,  30.813705 ]],

       ...,

       [[-18.594002 , -44.762398 ,  75.813705 ],
        [-1

In [10]:
train_image.shape

(3210, 224, 224, 3)

## Extract Features

#### VGG16 using fc6/relu

In [14]:
# Based on VGG16 architecture -> old paper(2015)
vgg16 = VGGFace(model='vgg16') # or VGGFace() as default

# There are 580070376 parameters in VGG16

In [15]:
vgg16.summary()

Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)     

In [16]:
# Create a new model that outputs the features from the 'fc6' layer
vgg16_fc6relu = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc6/relu').output)

In [17]:
from tensorflow.keras.utils import Sequence

In [18]:
# Use the model to extract features from the training images
vgg16_relu_train_features = vgg16_fc6relu.predict(train_image)

# Similarly, extract features from the testing images
vgg16_relu_test_features = vgg16_fc6relu.predict(test_image)


24/24 [==============================] - 48s 2s/step


In [19]:
# Save the arrays to local disk
np.save('vgg16_relu_train_features.npy', vgg16_relu_train_features)
np.save('vgg16_relu_test_features.npy', vgg16_relu_test_features)

#### VGG16 fc6

In [20]:
# Based on VGG16 architecture -> old paper(2015)
vgg16_notop = VGGFace(model='vgg16', include_top=False) # or VGGFace() as default

# There are 580070376 parameters in VGG16

In [21]:
vgg16_notop.summary()

Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv1_1 (Conv2D)            (None, None, None, 64)    1792      
                                                                 
 conv1_2 (Conv2D)            (None, None, None, 64)    36928     
                                                                 
 pool1 (MaxPooling2D)        (None, None, None, 64)    0         
                                                                 
 conv2_1 (Conv2D)            (None, None, None, 128)   73856     
                                                                 
 conv2_2 (Conv2D)            (None, None, None, 128)   147584    
                                                                 
 pool2 (MaxPooling2D)        (None, None, None, 128) 

There is no fc6 if exclude top layers

In [22]:
# Create a new model that outputs the features from the 'fc6' layer
vgg16_fc6 = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc6').output)

In [23]:
# Use the model to extract features from the training images
vgg16_train_features = vgg16_fc6.predict(train_image)

# Similarly, extract features from the testing images
vgg16_test_features = vgg16_fc6.predict(test_image)

24/24 [==============================] - 52s 2s/step


In [24]:
# Save the arrays to local disk
np.save('vgg16_train_features.npy', vgg16_train_features)
np.save('vgg16_test_features.npy', vgg16_test_features)

In [25]:
# Compare the arrays
print(np.array_equal(vgg16_train_features, vgg16_relu_train_features))


False


In [26]:
print(np.array_equal(vgg16_test_features, vgg16_relu_test_features))

False


#### VGG RESNET

In [38]:
train_image_resnet = np.load('train_image_resnet.npy')
test_image_resnet = np.load('test_image_resnet.npy')

In [39]:
# Based on RESNET50 architecture -> new paper(2017)
vgg_resnet = VGGFace(model='resnet50')
vgg_resnet.summary()


Model: "vggface_resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                  

|

In [40]:
# Create a new model that outputs the features from the 'avg_pool' layer
vgg_resnet_avg_pool = Model(inputs=vgg_resnet.input, outputs=vgg_resnet.get_layer('avg_pool').output)


In [41]:
# Use the model to extract features from the training images
resnet_train_features = vgg_resnet_avg_pool.predict(train_image_resnet)

# Similarly, extract features from the testing images
resnet_test_features = vgg_resnet_avg_pool.predict(test_image_resnet)

24/24 [==============================] - 15s 632ms/step


In [42]:
# Save the arrays to local disk
np.save('resnet_train_features.npy', resnet_train_features)
np.save('resnet_test_features.npy', resnet_test_features)

In [43]:
resnet_train_features.shape

(3210, 1, 1, 2048)

#### VGG SENET

In [11]:
# same with resnet since they both experence version preprocess
train_image_senet = np.load('train_image_resnet.npy')
test_image_senet = np.load('test_image_resnet.npy')

In [12]:
# Based on SENET50 architecture -> new paper(2019)
vgg_senet = VGGFace(model='senet50')
vgg_senet.summary()

Model: "vggface_senet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                   

In [13]:
# Create a new model that outputs the features from the 'avg_pool' layer
vgg_senet_avg_pool = Model(inputs=vgg_senet.input, outputs=vgg_senet.get_layer('avg_pool').output)

In [14]:
# save this model to local
vgg_senet_avg_pool.save('vgg_senet_avg_pool_model.h5')

In [16]:
from tensorflow.keras.models import load_model

In [17]:
vgg_senet_avg_pool = load_model('vgg_senet_avg_pool_model.h5')

In [18]:
# Use the model to extract features from the training images
senet_train_features = vgg_senet_avg_pool.predict(train_image_senet)

# Similarly, extract features from the testing images
senet_test_features = vgg_senet_avg_pool.predict(test_image_senet)

24/24 [==============================] - 16s 670ms/step


In [19]:
senet_train_features[:5]

array([[[[1.77558716e-02, 3.35661054e-01, 1.13176536e-02, ...,
          1.72436726e+00, 1.63295358e-01, 5.43274581e-02]]],


       [[[4.50852998e-02, 4.48929369e-02, 1.28231063e-01, ...,
          3.73613667e+00, 1.59444585e-02, 4.65025380e-02]]],


       [[[1.77555485e+01, 3.79963303e+00, 4.00157642e+00, ...,
          1.12641087e+01, 1.84112501e+00, 5.62081598e-02]]],


       [[[1.01859818e+01, 7.96744967e+00, 1.34816945e+00, ...,
          3.84514546e+00, 1.06552868e+01, 5.48046865e-02]]],


       [[[5.14905304e-02, 3.44009288e-02, 1.63564754e+00, ...,
          2.31483225e-02, 1.34842947e-01, 1.02118626e-01]]]],
      dtype=float32)

In [48]:
senet_train_features.shape

(3210, 1, 1, 2048)

In [49]:
# Save the arrays to local disk
np.save('senet_train_features.npy', senet_train_features)
np.save('senet_test_features.npy', senet_test_features)